In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "bigscience/bloom-1b7"  # замените на свою модель

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# загрузка в 8-bit (bitsandbytes автоматически используется)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",      # автоматически распределит слои по GPU/CPU
    load_in_8bit=True,      # <-- 8-bit
    torch_dtype=torch.float16,
)

# простая генерация
inputs = tokenizer("Привет! Расскажи анекдот:", return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.7)
print(tokenizer.batch_decode(out, skip_special_tokens=True)[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Привет! Расскажи анекдот:
Главное очевидное, что по писать на программе, пользуетсяся лишь один аналог, и потому что необходимость пользоваться необходимость неизвестна. Но вот очевидно, что в программе для этого необходимо писать количество разных аналоги. Поэтому, почему недостаточно проверять пользователь в качестве


In [3]:
# pip install -U bitsandbytes

# bnb

In [2]:
model.save_pretrained("quantized_model")
tokenizer.save_pretrained("quantized_model")

('quantized_model/tokenizer_config.json',
 'quantized_model/special_tokens_map.json',
 'quantized_model/tokenizer.json')

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(
    "quantized_model",
    device_map="auto",
    quantization_config=bnb
)

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


# torch


In [ ]:
torch.save(model.state_dict(), "model_quantized.pt")

In [ ]:
state = torch.load("model_quantized.pt", map_location="cpu")
model.load_state_dict(state)